This model is meant to forecast the outcome of the standings here: https://udisclive.com/players?t=standings&z=dgpt Logic for how to attribute points is here: https://udisc.com/blog/post/how-disc-golf-pro-tour-points-work-why-they-matter?fbclid=IwAR1VwYCkl7DCkgc93G5qujSDxCSWqg5HMWLv7dVhu_c4GXchW_P7fJO7MSo

To do so, I need to:

load in events, and details about those events
load in the players playing in those events, and details about their skill levels
generate N runs of a model that forecasts each player's results N times at each event they are entered in
use the generated results to assign points to the players
sum the points based on the DGPT rules
aggregate the ranks for each player at the end of the year (for example, see if Paige Pierce has a 90% chance of 1st place, 5% chance of 2nd place, etc.)
Things that inspired this / what the output should look like

538 soccer: https://projects.fivethirtyeight.com/soccer-predictions/champions-league/
my own attempt at this same thing but in google sheets: https://docs.google.com/spreadsheets/d/19IwKCt5DI77koId916DawdJd1mNM7RQi3Nzlpmsa23Q/edit#gid=0 (see ForecastFinal tab. This doc does ALL of the logic i want in here, but it's limited to N=50 runs due to how long it takes the formulas to update)

In [1]:
###Load Packages
import pandas as pd
import numpy as np

#Load Raw Event, Player, and Point Logic Data
Events = pd.read_csv('AllProjectedEvents.csv')
EventPlayers = pd.read_csv('AllProjectedEventPlayers08092021DGPTNTOnlyNoDGPTAfterIdlewild.csv')
PointsLogic = pd.read_csv('PointsLogicAllTours.csv')
pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 200)


#Prep Raw Data
Events['EventDate'] = pd.to_datetime(Events['EventDate'])
Events['EventID'] = Events.EventID.fillna(1)
Events['EventID'] = Events.EventID.astype(int)
EventPlayers['Rating'] = pd.to_numeric(EventPlayers['Rating'])
EventPlayers['PDGANumber'] = EventPlayers.PDGANumber.astype(int)

#Declare Variables
RatingPointsPerStroke = 6
NumberOfModelRuns = 1000
RoundStandardDeviation = 6.82

In [2]:
#Create a Mean Regression so that Events further out have more uncertainty
EventMeanRegression = Events[['EventID','EventDate']].drop_duplicates(subset=['EventID','EventDate'])
EventMeanRegression['Today'] = pd.to_datetime("now")
EventMeanRegression['TimeToEvent'] = EventMeanRegression['EventDate'] - EventMeanRegression['Today']
EventMeanRegression['TimeToEventNumber'] = pd.to_numeric(EventMeanRegression['TimeToEvent'])/(1000000000*60*60*24)
EventMeanRegression['EventMeanRegressionValue'] = np.log10(np.absolute(EventMeanRegression['TimeToEventNumber']))
###Placeholder - if TimeToEvent Negative, then EventMeanRegressionValue = 0
###Placeholder - create a boolean to check if events have passed
EventMeanRegression = EventMeanRegression[['EventID','EventDate','EventMeanRegressionValue']].sort_values(by=['EventDate'])
###EventMeanRegression

In [3]:
###Calculate Average Rating for Each Event and Division
EventAverageRating = EventPlayers[['EventID','Division','Rating']]
EventDivisionAverageRatingValue = EventAverageRating.groupby(['EventID','Division'])['Rating'].mean()
EventAverageRating = EventPlayers[['EventID','Division']]
EventAverageRating = pd.merge(EventAverageRating,EventDivisionAverageRatingValue,on = ['EventID','Division'],how='left')
EventAverageRating = EventAverageRating.drop_duplicates()
EventAverageRating = EventAverageRating.rename(columns={"Rating":"EventDivisionAverageRatingValue"}).sort_values(by=['EventID','Division'])
###EventAverageRating

In [4]:
###Calculate Expected Score for a Single Round for Each Player
SingleRoundExpectedScores = EventPlayers[['PDGANumber','EventID','Division','Rating']]
SingleRoundExpectedScores = pd.merge(SingleRoundExpectedScores,EventMeanRegression, on = ['EventID'],how = 'left')
SingleRoundExpectedScores = pd.merge(SingleRoundExpectedScores,EventAverageRating, on = ['EventID','Division'],how = 'left')
SingleRoundExpectedScores['SingleRoundExpectedScoreValue'] = -1*((SingleRoundExpectedScores['Rating']-SingleRoundExpectedScores['EventDivisionAverageRatingValue'])/(RatingPointsPerStroke+SingleRoundExpectedScores['EventMeanRegressionValue']))
SingleRoundExpectedScores = SingleRoundExpectedScores[['PDGANumber','EventID','Division','SingleRoundExpectedScoreValue']].sort_values(by=['EventID','Division','SingleRoundExpectedScoreValue'])
###SingleRoundExpectedScores

In [5]:
RoundRandomScores = pd.concat([SingleRoundExpectedScores for i in range(NumberOfModelRuns)], ignore_index=True)
RoundRandomScores['ModelRunNumber'] = RoundRandomScores.groupby(['EventID','PDGANumber','Division'])['PDGANumber'].rank(method='first').astype(int)
RoundRandomScores = pd.merge(RoundRandomScores,Events[['EventID','EventLength']], on = ['EventID'],how = 'left')
RoundRandomScores = RoundRandomScores.iloc[np.arange(len(RoundRandomScores)).repeat(RoundRandomScores['EventLength'])]
RoundRandomScores['RoundNumber']= RoundRandomScores.groupby(['EventID','PDGANumber','ModelRunNumber','Division'])['PDGANumber'].rank(method='first').astype(int)
RoundRandomScores['RoundStandardDeviation']= RoundStandardDeviation
RoundRandomScores['RoundRandomScoreValue'] = np.random.normal(
        RoundRandomScores['SingleRoundExpectedScoreValue'].values,
        RoundRandomScores['RoundStandardDeviation'].values).round(0).astype(int)
RoundRandomScores = RoundRandomScores[['PDGANumber','EventID','Division','ModelRunNumber','RoundRandomScoreValue']].sort_values(by=['EventID','Division','ModelRunNumber','RoundRandomScoreValue'])
###RoundRandomScores
###Paul McBeth is always expected to score under par so i'm doing something right
###RoundRandomScores.query('PDGANumber == "27523"').hist(column='RoundRandomScoreValue', by='EventID')

In [6]:
EventRandomScores = RoundRandomScores.groupby(['EventID','PDGANumber','ModelRunNumber','Division'],as_index=False).sum('RoundRandomScoreValue')
EventRandomScores = EventRandomScores.rename(columns={"RoundRandomScoreValue":"EventRandomScoreValue"}).sort_values(by=['EventID','Division','ModelRunNumber','EventRandomScoreValue'])
###EventRandomScores
###Check on Paul McBeth
###EventRandomScores.query('PDGANumber == "27523"').hist(column='EventRandomScoreValue', by='EventID')
###EventRandomScores.query('EventID == "52009" & ModelRunNumber == "1" & Division == "MPO"').sort_values(by=['EventRandomScoreValue'])

In [7]:
EventRunRanks = EventRandomScores
EventRunRanks['EventRankBeforePlayoff'] = EventRunRanks.groupby(['EventID','ModelRunNumber','Division'])['EventRandomScoreValue'].rank(method='min').astype(int)
EventRunRanks = EventRunRanks[['EventID','PDGANumber','ModelRunNumber','EventRankBeforePlayoff','Division']].sort_values(by=['EventID','Division','ModelRunNumber','EventRankBeforePlayoff'])
###EventRunRanks.query('EventID == "52009" & ModelRunNumber == "1" & Division == "MPO"').sort_values(by=['EventRankBeforePlayoff'])

In [8]:
###Ridiculous amount of code to break ties in playoffs. Two things it should do that it doesn't : account for strength of player, account for the low % chance that a playoff can't occur
FirstPlaceTies = EventRunRanks
NumberOfTiesBeforePlayoff = FirstPlaceTies.groupby(['EventRankBeforePlayoff','EventID','ModelRunNumber','Division'],as_index=False)['EventRankBeforePlayoff'].size()
FirstPlaceTies = pd.merge(EventRunRanks,NumberOfTiesBeforePlayoff, on = ['EventID','EventRankBeforePlayoff','ModelRunNumber','Division'],how = 'right')
FirstPlaceTies = FirstPlaceTies.rename(columns={"size":"NumberOfFirstPlaceTies"})
FirstPlaceTies = FirstPlaceTies[FirstPlaceTies['NumberOfFirstPlaceTies']>1]
FirstPlaceTies = FirstPlaceTies[FirstPlaceTies['EventRankBeforePlayoff']==1]
FirstPlaceTies['PlayoffWinnerDetermination'] = np.random.rand(len(FirstPlaceTies.index))
FirstPlaceTies['EventRankAfterPlayoffBeforeSecondPlaceTies']= FirstPlaceTies.groupby(['EventID','ModelRunNumber','Division'])['PlayoffWinnerDetermination'].rank(method='first').astype(int)
FirstPlaceTies['EventRankAfterPlayoff']= FirstPlaceTies['EventRankAfterPlayoffBeforeSecondPlaceTies'].astype(int)
FirstPlaceTies['EventRankAfterPlayoff'] = np.where((FirstPlaceTies.EventRankAfterPlayoffBeforeSecondPlaceTies >= 2),2,FirstPlaceTies.EventRankAfterPlayoff).astype(int)
FirstPlaceTies = FirstPlaceTies[['EventID','PDGANumber','ModelRunNumber','EventRankAfterPlayoff','Division']].sort_values(by=['EventID','ModelRunNumber','Division','EventRankAfterPlayoff'])
###FirstPlaceTies.query('EventID == "52009" & ModelRunNumber == "1" & Division == "MPO"').sort_values(by=['EventRankAfterPlayoff'])
###FirstPlaceTies.hist(column='RankAfterPlayoff')

In [9]:
EventRunRanks = pd.merge(EventRunRanks,FirstPlaceTies, on = ['EventID','PDGANumber','ModelRunNumber','Division'],how = 'left')
EventRunRanks.EventRankAfterPlayoff.fillna(EventRunRanks.EventRankBeforePlayoff, inplace = True)
EventRunRanks['EventRankAfterPlayoff'] = EventRunRanks.EventRankAfterPlayoff.astype(int)
###EventRunRanks['EventRankForPoints']
EventRunRanks = EventRunRanks[['EventID','PDGANumber','ModelRunNumber','Division','EventRankAfterPlayoff']]
###EventRunRanks.query('EventID == "52009" & ModelRunNumber == "1" & Division == "MPO"').sort_values(by=['EventRankAfterPlayoff'])

In [10]:
### Calcluate Points for Each Run
RankProjectedAndActual = pd.merge(EventRunRanks,Events[['EventID','Division']], on = ['EventID','Division'],how = 'left')
###Final Ranks are completed tournaments
RankProjectedAndActual = pd.merge(RankProjectedAndActual,EventPlayers[['EventID','Division','PDGANumber','FinalRank']], on = ['EventID','Division','PDGANumber'],how = 'left')
RankProjectedAndActual.FinalRank = np.where(RankProjectedAndActual.FinalRank.isnull(),RankProjectedAndActual.EventRankAfterPlayoff,RankProjectedAndActual.FinalRank).astype(int)
RankProjectedAndActual['LookupRankForPoints'] = RankProjectedAndActual.groupby(['EventID','ModelRunNumber','Division'])['FinalRank'].rank(method='first').astype(int)
RankProjectedAndActual = RankProjectedAndActual[['EventID','PDGANumber','ModelRunNumber','Division','FinalRank','LookupRankForPoints']].sort_values(by=['EventID','ModelRunNumber','Division','LookupRankForPoints'])
###RankProjectedAndActual.query('EventID == "52009" & ModelRunNumber == "1" & Division == "MPO"').sort_values(by=['LookupRankForPoints'])

In [11]:
###merge the number of ties back into the thing
###find the number of ties
NumberOfTies = RankProjectedAndActual.groupby(['FinalRank','EventID','ModelRunNumber','Division'],as_index=False)['FinalRank'].size()
NumberOfTies.rename(columns = {'size':'NumberOfTies'}, inplace = True)
###NumberOfTies

In [12]:
RankProjectedAndActual = pd.merge(RankProjectedAndActual,NumberOfTies, on = ['EventID','FinalRank','ModelRunNumber','Division'],how = 'left')
###RankProjectedAndActual

In [13]:
###Need to merge in the points before i sum, that's the whole point of the exercise
PointsLookup = RankProjectedAndActual[['EventID','PDGANumber','ModelRunNumber','Division','FinalRank','LookupRankForPoints','NumberOfTies']].sort_values(by=['EventID','Division','ModelRunNumber','FinalRank'])
PointsLookup = pd.merge(PointsLookup,Events[['Division','EventID','EventType','EventTour']],on = ['Division','EventID'], how = 'left')
PointsLookup = pd.merge(PointsLookup,PointsLogic, on = ['EventTour','LookupRankForPoints'], how = 'left')
###PointsLookup

In [14]:
TieDivision = PointsLookup.groupby(['FinalRank','EventID','ModelRunNumber','NumberOfTies','Division','EventType','EventTour'],as_index=False).sum()
TieDivision['PointsDividedBetweenTies'] = TieDivision['Points']/TieDivision['NumberOfTies']
TieDivision['PointsDividedBetweenTies'] = np.where(TieDivision['EventType'] == 'Silver', TieDivision['PointsDividedBetweenTies']/4,TieDivision['PointsDividedBetweenTies'])
TieDivision = TieDivision.sort_values(by=['EventID','Division','ModelRunNumber','PointsDividedBetweenTies'])
TieDivision= TieDivision[['EventID','EventTour','EventType','FinalRank','ModelRunNumber','Division','PointsDividedBetweenTies']]
###TieDivision

In [15]:
PointsProjectedAndActual = pd.merge(RankProjectedAndActual,TieDivision,on = ['Division','EventID','FinalRank','ModelRunNumber'], how = 'left')
PointsProjectedAndActual = PointsProjectedAndActual[['EventID','PDGANumber','ModelRunNumber','Division','EventTour','EventType','FinalRank','PointsDividedBetweenTies']]
###PointsProjectedAndActual

In [16]:
TourEventRankings = PointsProjectedAndActual
TourEventRankings['EliteSeriesRank'] = TourEventRankings[TourEventRankings.EventType == 'Elite'].groupby(['PDGANumber','ModelRunNumber'])['PointsDividedBetweenTies'].rank(method='first',ascending = False)
TourEventRankings['EliteSeriesRank'] = TourEventRankings['EliteSeriesRank'].fillna(100)
TourEventRankings['EliteSeriesRank'] = TourEventRankings.EliteSeriesRank.astype(int)

TourEventRankings['SilverSeriesRank'] = TourEventRankings[TourEventRankings.EventType == 'Silver'].groupby(['PDGANumber','ModelRunNumber'])['PointsDividedBetweenTies'].rank(method='first',ascending = False)
TourEventRankings['SilverSeriesRank'] = TourEventRankings['SilverSeriesRank'].fillna(100)
TourEventRankings['SilverSeriesRank'] = TourEventRankings.SilverSeriesRank.astype(int)

###Careful with NT - the Music City Open must count therefore it's EventType = Last. Otherwise top 4 of 5 count
TourEventRankings['NTRank'] = TourEventRankings[TourEventRankings.EventType == 'NT'].groupby(['PDGANumber','ModelRunNumber'])['PointsDividedBetweenTies'].rank(method='first',ascending = False)
TourEventRankings['NTRank'] = TourEventRankings['NTRank'].fillna(100)
TourEventRankings['NTRank'] = TourEventRankings.NTRank.astype(int)

TourEventRankings['PDPTRank'] = TourEventRankings[TourEventRankings.EventType == 'PDPT'].groupby(['PDGANumber','ModelRunNumber'])['PointsDividedBetweenTies'].rank(method='first',ascending = False)
TourEventRankings['PDPTRank'] = TourEventRankings['PDPTRank'].fillna(100)
TourEventRankings['PDPTRank'] = TourEventRankings.PDPTRank.astype(int)


TourEventRankings['KeepScore'] = np.where((TourEventRankings['EliteSeriesRank']<=8)|(TourEventRankings['SilverSeriesRank']<=3)|(TourEventRankings['NTRank']<=4)|(TourEventRankings['PDPTRank']<=4),'Yes','No')
###TourEventRankings = TourEventRankings[['EventID','PDGANumber','ModelRunNumber','Division','EventTour','FinalRank','PointsDividedBetweenTies','KeepScore']]
###TourEventRankings.query('PDGANumber == "66842" & ModelRunNumber == "1" & EventTour == "DGPT"')
###TourEventRankings.dtypes

In [17]:
ModelRunScore = TourEventRankings[['PDGANumber','ModelRunNumber','Division','EventTour','PointsDividedBetweenTies','KeepScore']]
ModelRunScore = ModelRunScore[ModelRunScore.KeepScore =='Yes'].groupby(['PDGANumber','ModelRunNumber','Division','EventTour'],as_index=False).sum()
ModelRunScore = ModelRunScore.rename(columns={"PointsDividedBetweenTies":"TotalTourDivisionPoints"}).sort_values(by=['ModelRunNumber','Division','EventTour','TotalTourDivisionPoints'])

###ModelRunScore.query('PDGANumber == "17295"')

In [18]:
ModelRunStandings = ModelRunScore
ModelRunStandings['FinalStandings'] = ModelRunStandings.groupby(['ModelRunNumber','Division','EventTour'])['TotalTourDivisionPoints'].rank(ascending=False,method='min').astype(int)

###Create Subset of just unique players to finally add name back in
EventPlayersUnique = EventPlayers.drop_duplicates(subset=['PDGANumber','Name'])
EventPlayersUnique = EventPlayersUnique[['PDGANumber','Name']]

ModelRunStandings = pd.merge(ModelRunStandings,EventPlayersUnique, on = ['PDGANumber'], how = 'left')
ModelRunStandings = ModelRunStandings[['Name','PDGANumber','ModelRunNumber','Division','EventTour','FinalStandings']]
###ModelRunStandings

In [19]:
Visual = ModelRunStandings[['PDGANumber','Division','EventTour','FinalStandings']]

AverageFinish = Visual.groupby(['PDGANumber','Division','EventTour'])['FinalStandings'].mean().reset_index().rename(columns={"FinalStandings":"AverageFinish"})

TopEight = Visual[Visual.FinalStandings <= 8]
TopEight = TopEight[['PDGANumber','Division','EventTour']].value_counts().to_frame(name="TopEight")

TopSixteen =  Visual[Visual.FinalStandings <= 16]
TopSixteen = TopSixteen[['PDGANumber','Division','EventTour']].value_counts().to_frame(name="TopSixteen")

TopThirtyTwo =  Visual[Visual.FinalStandings <= 32]
TopThirtyTwo = TopThirtyTwo[['PDGANumber','Division','EventTour']].value_counts().to_frame(name="TopThirtyTwo")

In [20]:
EventPlayerTour = pd.merge(EventPlayers,Events, on = ['Division','EventID'],how = 'left')
EventPlayerTour = EventPlayerTour[['Name','EventID','PDGANumber','FinalRank','Division','EventType','EventTour','EventDate']]

SilverSeriesStops = EventPlayerTour[EventPlayerTour.EventType == "Silver"].groupby(['PDGANumber','Division','EventType','EventTour']).size().to_frame(name="SilverSeriesStops")
EliteSeriesStops = EventPlayerTour[EventPlayerTour.EventType == "Elite"].groupby(['PDGANumber','Division','EventType','EventTour']).size().to_frame(name="EliteSeriesStops")
PDPTStops = EventPlayerTour[EventPlayerTour.EventType == "PDPT"].groupby(['PDGANumber','Division','EventType','EventTour']).size().to_frame(name="PDPTStops")
NTStopsBeforeMCO = EventPlayerTour[EventPlayerTour.EventType == "NT"].groupby(['PDGANumber','Division','EventType','EventTour']).size().to_frame(name="NTStopsBeforeMCO")
###MinimumPoints
###MaximumPoints
###NextDroppedScoreEliteNTPDPT
###NextDroppedScoreSilver

UniqueEventPlayerDivisionTour = EventPlayerTour[['Name','PDGANumber','Division','EventTour']].drop_duplicates()

UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,TopEight,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,TopSixteen,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,TopThirtyTwo,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,AverageFinish,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,SilverSeriesStops,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,EliteSeriesStops,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,PDPTStops,on = ['PDGANumber','Division','EventTour'],how = 'left')
UniqueEventPlayerDivisionTour = pd.merge(UniqueEventPlayerDivisionTour,NTStopsBeforeMCO,on = ['PDGANumber','Division','EventTour'],how = 'left')

UniqueEventPlayerDivisionTour = UniqueEventPlayerDivisionTour.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('.'))

UniqueEventPlayerDivisionTour = UniqueEventPlayerDivisionTour.astype({'TopEight': 'int','TopSixteen': 'int','TopThirtyTwo': 'int','SilverSeriesStops': 'int','EliteSeriesStops': 'int','PDPTStops': 'int','NTStopsBeforeMCO': 'int'})
UniqueEventPlayerDivisionTour.to_csv('FinalResultsAfterIdlewildNoPaul08112021.csv')

In [21]:
###FinalDGPTMPOResultsAggregated = ModelRunStandings[ModelRunStandings.EventTour == 'DGPT']
###FinalDGPTMPOResultsAggregated = FinalDGPTMPOResultsAggregated[FinalDGPTMPOResultsAggregated.Division == 'MPO']
###FinalDGPTMPOResultsAggregated = FinalDGPTMPOResultsAggregated[['Name','PDGANumber','ModelRunNumber','FinalStandings']].sort_values(by=['FinalStandings','ModelRunNumber'])
###FinalDGPTMPOResultsAggregated.query('PDGANumber == "17295"')
###FinalDGPTMPOResultsAggregated = pd.pivot_table(FinalDGPTMPOResultsAggregated,index=['PDGANumber','Name'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalDGPTFPOResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalNTMPOResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalNTFPOResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTMPOResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTFPOResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTMP40ResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTFP40ResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTMP50ResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)
###FinalPDPTMP60ResultsAggregated = pd.pivot_table(ModelRunStandings,index=['PDGANumber'],columns=['FinalStandings'],aggfunc='count',fill_value =0)

###FinalDGPTMPOResultsAggregated.to_csv('FinalDGPTMPOResultsAggregates08092021.csv')
###FinalDGPTMPOResultsAggregated

###FinalDGPTFPOResultsAggregated = ModelRunStandings[ModelRunStandings.EventTour == 'DGPT']
###FinalDGPTFPOResultsAggregated = FinalDGPTFPOResultsAggregated[FinalDGPTFPOResultsAggregated.Division == 'FPO']
###FinalDGPTFPOResultsAggregated = FinalDGPTFPOResultsAggregated[['Name','PDGANumber','ModelRunNumber','FinalStandings']].sort_values(by=['FinalStandings','ModelRunNumber'])
###FinalDGPTFPOResultsAggregated = pd.pivot_table(FinalDGPTFPOResultsAggregated,index=['PDGANumber','Name'],columns=['FinalStandings'],aggfunc='count',fill_value =0)

###FinalDGPTFPOResultsAggregated.to_csv('FinalDGPTFPOResultsAggregates08092021.csv')
###FinalDGPTFPOResultsAggregated